In [1]:
import os
import pickle


hand_landmarks_dict = {
    0: "WRIST",
    1: "THUMB_CMC",
    2: "THUMB_MCP",
    3: "THUMB_IP",
    4: "THUMB_TIP",
    5: "INDEX_FINGER_MCP",
    6: "INDEX_FINGER_PIP",
    7: "INDEX_FINGER_DIP",
    8: "INDEX_FINGER_TIP",
    9: "MIDDLE_FINGER_MCP",
    10: "MIDDLE_FINGER_PIP",
    11: "MIDDLE_FINGER_DIP",
    12: "MIDDLE_FINGER_TIP",
    13: "RING_FINGER_MCP",
    14: "RING_FINGER_PIP",
    15: "RING_FINGER_DIP",
    16: "RING_FINGER_TIP",
    17: "PINKY_MCP",
    18: "PINKY_PIP",
    19: "PINKY_DIP",
    20: "PINKY_TIP"
}

# static/landmarks 폴더의 절대 경로를 계산
landmarks_path = os.path.join('guide_landmarks.pkl')

# 가이드 영상의 좌표를 미리 로드
with open(landmarks_path, 'rb') as f:
    guide_landmarks = pickle.load(f)

In [2]:
import numpy as np

In [3]:
from dtw import accelerated_dtw
from scipy.spatial.distance import euclidean, cosine

def compare_joint(series_a, series_b):
    # frame_num, joint_num, 3(x,y,z)
    sim = 0
    for joint_a, joint_b in zip(series_a, series_b):
        dist, _, _, _ = accelerated_dtw(joint_a, joint_b, dist="matching")
        sim += dist
    sim /= len(series_a)
    return sim

In [12]:
def compare_hand(guide_landmarks, result_landmarks):
    guide_landmarks = np.array(guide_landmarks)
    result_landmarks = np.array(result_landmarks)
    
    dist_joints = []
    max_joint = [None,None]
    
    for i in range(2):
        guide_hand = guide_landmarks[:,i,:,:]
        result_hand = result_landmarks[:,i,:,:]
        hand_joints = []
        
        for joint in range(21):
            series_a = guide_hand[:,joint,:]
            series_b = result_hand[:,joint,:]
            dist = compare_joint(series_a, series_b)
            hand_joints.append(dist)
            
        dist_joints.append(hand_joints)
        if max(hand_joints)>0.1:
            max_idx=np.argmax(hand_joints)
            max_joint[i] = hand_landmarks_dict[max_idx]
    
    return max(dist_joints[0]), max_joint

In [13]:
compare_hand(np.array(guide_landmarks), np.array(guide_landmarks))

(0.0, [None, None])